In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb

# 모델 및 클라이언트 초기화
model = SentenceTransformer('all-MiniLM-L6-v2')
client = chromadb.PersistentClient()

# 기존 컬렉션 삭제 후 생성
if "crawled_data" in [col.name for col in client.list_collections()]:
    client.delete_collection(name="crawled_data")
collection = client.create_collection(name="crawled_data")

# 문서 데이터 추가 - 상세 설명 포함(예시 데이터)
documents = [
    "K-패스(구. 알뜰교통카드) - 저소득 청년 대상 교통비 할인 지원 프로그램",
    "청년도약계좌 - 청년 대상 저축 장려 프로그램으로, 매칭 지원을 통해 저축을 돕습니다.",
    "무료소송지원 - 법적 어려움이 있는 청년들을 위한 무료 소송 지원 서비스",
    "서울 고립, 은둔 청년 지원 프로그램 - 고립된 청년들에게 사회 복귀 지원을 제공하는 프로그램",
]

# 임베딩 생성 및 데이터베이스에 저장
for idx, doc in enumerate(documents):
    embedding = model.encode([doc])[0]
    collection.add(
        documents=[doc],
        embeddings=[embedding],
        ids=[f"doc_{idx+1}"]
    )

print("데이터 저장이 완료되었습니다.")